In [1]:
from multi_agents.tools.insta_tools import *

In [2]:
from multi_agents.constants.constants import COOKIES
COOKIES


{'sessionid': '62433026643:l3mmUYSJJttHBo:5:AYdYFZ5JDSfjGj-kx0vmLp8IUvJQVGV95byDQLK4Pg',
 'ds_user_id': '62433026643',
 'csrftoken': 'ytAcHc3ynwuG61P1UqIayswla4B0eymh'}

In [8]:
import requests, random, time
from urllib.parse import unquote

def ig_cookie_header(c: dict) -> str:
    # decode any percent-encoded cookie values and strip stray quotes
    def clean(v: str) -> str:
        if not isinstance(v, str): return v
        v = unquote(v).strip()
        if len(v) >= 2 and v[0] in "\"'" and v[-1] == v[0]:
            v = v[1:-1]
        return v.replace("\\054", ",")  # DevTools escape → comma
    need = {
        "sessionid": clean(c.get("sessionid","")),
        "ds_user_id": clean(c.get("ds_user_id","")),
        "csrftoken": clean(c.get("csrftoken","")),
        # These help a lot; add if you have them from the same browser session:
        "rur": clean(c.get("rur","ODN")),          # fallback region token
        "mid": clean(c.get("mid","")),
        "ig_did": clean(c.get("ig_did","")),
        "ig_nrcb": clean(c.get("ig_nrcb","1")),
    }
    # sanity: sessionid must be present and decoded (no %3A)
    assert ":" in need["sessionid"] and "%3A" not in need["sessionid"], \
        f"Bad sessionid format: {need['sessionid']!r}"
    # build Cookie header
    return "; ".join([f"{k}={v}" for k,v in need.items() if v])

def fetch_profile(username: str, COOKIES: dict):
    s = requests.Session()
    # Base headers
    s.headers.update({
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/127.0.0.0 Safari/537.36"),
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.instagram.com/",
        "X-IG-App-ID": "936619743392459",
        "X-ASBD-ID": "129477",
        "X-IG-WWW-Claim": "0",
        "X-Requested-With": "XMLHttpRequest",
    })
    # Prime visit (gets/refreshes csrftoken/rur on this IP)
    s.get("https://www.instagram.com/", timeout=20)
    # Force Cookie + CSRF headers explicitly for the API call
    cookie_header = ig_cookie_header(COOKIES)
    s.headers["Cookie"] = cookie_header
    # Make X-CSRFToken match the cookie we’re sending
    # (prefer the post-prime one if IG set a new token)
    csrf = COOKIES.get("csrftoken") or s.cookies.get("csrftoken", "")
    s.headers["X-CSRFToken"] = csrf

    time.sleep(random.uniform(1.6, 3.0))
    url = f"https://www.instagram.com/api/v1/users/web_profile_info/?username={username}"
    r = s.get(url, timeout=25)
    if r.status_code != 200:
        return {
            "status": r.status_code,
            "text": r.text[:400],
            "sent_X_CSRFToken": s.headers.get("X-CSRFToken"),
            "sent_Cookie_has_sessionid": ("sessionid=" in cookie_header),
        }
    data = r.json().get("data", {}).get("user")
    if not data:
        return {"error": f"user '{username}' not found"}
    return {
        "user_id": data.get("id"),
        "full_name": data.get("full_name"),
        "followers": data.get("edge_followed_by", {}).get("count"),
        "following": data.get("edge_follow", {}).get("count"),
        "post_count": data.get("edge_owner_to_timeline_media", {}).get("count"),
        "is_private": data.get("is_private"),
        "is_verified": data.get("is_verified"),
        "profile_pic_url": data.get("profile_pic_url_hd"),
        "external_url": data.get("external_url"),
    }


In [9]:
COOKIES = {
    "sessionid": "62433026643:l3mmUYSJJttHBo:5:AYdYFZ5JDSfjGj-kx0vmLp8IUvJQVGV95byDQLK4Pg",
    "ds_user_id": "62433026643",
    "csrftoken": "ytAcHc3ynwuG61P1UqIayswla4B0eymh",
    # Add these if you have them from the SAME browser session:
    # "rur": "ODN,62433026643,0178943944924:01f:...",  <-- no quotes, no \054
    # "mid": "aGqB0QAALAAEPMazH89y4d7uKnEpv",
    # "ig_did": "0169D0B4-DBD4-40D5-A92E-E517E2C93EF2",
    # "ig_nrcb": "1",
}
print(fetch_profile("simowows", COOKIES))


{'status': 401, 'text': '{"message":"Please wait a few minutes before you try again.","require_login":true,"igweb_rollout":true,"status":"fail"}', 'sent_X_CSRFToken': 'ytAcHc3ynwuG61P1UqIayswla4B0eymh', 'sent_Cookie_has_sessionid': True}


get_insta_user_info

In [3]:
from multi_agents.tools.insta_tools import get_instagram_user_info
uname = "simowows"
info = get_instagram_user_info.invoke(uname)

In [4]:
info

{'error': "HTTP error for 'simowows': Status 401. Check cookies or username."}

get_user_id

In [5]:
user_id = info["user_id"]
user_id


KeyError: 'user_id'

get_user_posts

In [ ]:
from multi_agents.tools.insta_tools import get_instagram_user_info, get_instagram_user_posts

username = "simowows"
info = get_instagram_user_info.invoke(username)
user_id = info["user_id"]

posts = get_instagram_user_posts.invoke(user_id, limit=20, username=username)  # <-- pass username for Referer


KeyError: 'user_id'

In [ ]:
posts

{'error': 'Network or HTTP error fetching posts: 401 Client Error: Unauthorized for url: https://www.instagram.com/api/v1/feed/user/54927492788/?count=12'}

download_image

In [ ]:
download_image('https://www.superprof.com/name-hassan-and-passionate-about-the-french-language-since-2014-have-been-teaching-the-secondary-school-level-and.html')

'c:\\Users\\hafss\\OneDrive\\Desktop\\Final\\multi-agent-system-for-identity-revelalion-and-reporting\\notebooks\\profile_pic.jpg'

In [ ]:
from multi_agents.tools.insta_tools import get_instagram_user_info
from multi_agents.tools.vision_tools import compare_profile_pictures_tool

username = "simowows"
ig_info   = get_instagram_user_info.invoke(username)
ig_avatar = posts["image_url"]
print(ig_avatar)
airbnb_avatar = "https://a0.muscache.com/im/pictures/user/User/original/e925659d-c53f-4d10-a029-65113eb66ace.jpeg?im_w=720"  # <- your Airbnb profile image URL

KeyError: 'image_url'

In [ ]:
# list form
result = compare_profile_pictures_tool.invoke({
    "image_sources": [ig_avatar, airbnb_avatar]
})
print(result)


{'confidence': 'Low', 'justification': 'The two images show individuals with distinct facial features and hairstyles. The first person has a beard, curly hair, and is wearing a white shirt, while the second person is wearing a cap and has a different facial structure and hairstyle. The backgrounds are also significantly different, with one being indoors and the other in a desert setting. These differences make it unlikely that the images depict the same person.', 'success': True}


In [ ]:
from multi_agents.tools.insta_tools import get_instagram_user_info
from multi_agents.tools.vision_tools import compare_profile_pictures_tool

username = "simowows"
ig_info   = get_instagram_user_info.invoke(username)
ig_avatar = ig_info["profile_pic_url"]
print(ig_avatar)
airbnb_avatar = "https://a0.muscache.com/im/pictures/user/User/original/e925659d-c53f-4d10-a029-65113eb66ace.jpeg?im_w=720"  # <- your Airbnb profile image URL

In [ ]:
from multi_agents.tools.insta_tools import get_instagram_user_info, get_instagram_user_posts
from multi_agents.tools.vision_tools import compare_profile_pictures_tool

username   = "simowows"
airbnb_img = "https://a0.muscache.com/im/pictures/user/User/original/e925659d-c53f-4d10-a029-65113eb66ace.jpeg?im_w=720"

info    = get_instagram_user_info.invoke(username)
user_id = info["user_id"]
posts   = get_instagram_user_posts.invoke(user_id, limit=20, username=username)

# ---- pick a post image first, else fall back to profile pic
ig_img = None
if isinstance(posts, list) and posts:
    for p in posts:
        if isinstance(p, dict) and p.get("image_url"):
            ig_img = p["image_url"]
            print("Using IG POST image from:", p.get("shortcode") or p.get("id"))
            break

if not ig_img:
    ig_img = info["profile_pic_url"]
    print("No post image found; using IG PROFILE pic.")

print("IG image used:", ig_img)
print("Airbnb image:", airbnb_img)

result = compare_profile_pictures_tool.invoke({
    "image_sources": [ig_img, airbnb_img]
})
print(result)


No post image found; using IG PROFILE pic.
IG image used: https://instagram.frba4-1.fna.fbcdn.net/v/t51.2885-19/326117053_728108528743414_4085775869464108815_n.jpg?stp=dst-jpg_s320x320_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6InByb2ZpbGVfcGljLmRqYW5nby4xMDgwLmMyIn0&_nc_ht=instagram.frba4-1.fna.fbcdn.net&_nc_cat=110&_nc_oc=Q6cZ2QHngNKQvvo7oMdBiAAq90hte0ez-KbTaznEbsKHuKIzrimcsBhi70nmqM2lFx7DFbM&_nc_ohc=A65QcfPbcWYQ7kNvwE6SWKd&_nc_gid=4CsjyTsgivTPGES1DxA-3g&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AfY4itgXsrPnlw8NF0AhEVhbSalJy8vrQFRznhdlkz1jBw&oe=68CBC584&_nc_sid=8b3546
Airbnb image: https://a0.muscache.com/im/pictures/user/User/original/e925659d-c53f-4d10-a029-65113eb66ace.jpeg?im_w=720
{'confidence': 'Low', 'justification': 'The two images show individuals with distinct facial features, hairstyles, and attire. The first person has a beard, curly hair, and is wearing a white shirt, while the second person is wearing a cap and has a different facial structure and expression. The backgrounds are also vastly diff

In [ ]:
print(type(posts), getattr(posts, '__len__', lambda: 'n/a')())
print(posts if not isinstance(posts, list) else posts[:2])


<class 'dict'> 1
{'error': 'Network or HTTP error fetching posts: 401 Client Error: Unauthorized for url: https://www.instagram.com/api/v1/feed/user/54927492788/?count=12'}


In [2]:
# 1) Scrape & load
from multi_agents.tools.search_tools import instagram_scrape_and_load
instagram_scrape_and_load.invoke({"username_or_url":"https://www.instagram.com/humansofny/","results":"posts","results_limit":5})

# 2) Query
from multi_agents.tools.insta_tools import instagram_db_get_posts, instagram_db_get_images, instagram_db_get_comments
posts = instagram_db_get_posts.invoke({"username":"humansofny","limit":5})
first_post_id = posts[0]["id"]
imgs  = instagram_db_get_images.invoke({"post_id": first_post_id})
comms = instagram_db_get_comments.invoke({"post_id": first_post_id,"limit":20})


INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
c:\Users\hafss\OneDrive\Desktop\Final\multi-agent-system-for-identity-revelalion-and-reporting\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
[apify.instagram-scraper runId:nTSZTL8Gr3OLTQPMR] -> Status: RUNNING, Message: 
[apify.instagram-scraper runId:nTSZTL8Gr3OLTQPMR] -> 2025-09-17T14:03:48.360Z ACTOR: Pulling Docker image of build ZP2X6OZpcYd4letWr from registry.
[apify.instagram-scraper runId:nTSZTL8Gr3OLTQPMR] -> 2025-09-17T14:03:48.362Z ACTOR: Creating Docker container.
[apify.instagram-scraper


>>> Connecting to DuckDB and creating final tables directly...
    Database file will be at: C:\Users\hafss\OneDrive\Desktop\Final\multi-agent-system-for-identity-revelalion-and-reporting\notebooks\databases\instagram\instagram_data.duckdb
    - Creating 'posts' table with Primary Key...
    - Creating 'comments' table...
    - Creating 'images' table...

    Verification successful:
    - Loaded 5 posts.
    - Loaded 33 comments.
    - Loaded 25 images.
    Database connection closed.
